# 영화진흥위원회 OpenAPI 연동
- `JSON`을 불러와서 `excel`로 저장
- OPEN API같은 경우 하루에 연동할 수 있는 수가 제한되어 있다
    - 영화진흥위원회 같은 경우는 하루에 3000번만 연동 가능, 네이버는 2만번
- https://www.kofic.or.kr/
    - 해당 사이트에 가입해야지 Open API를 사용 가능
    - 로그인 후 키 발급/관리 에서 키(API 연동 키) 발급 받기
    - OPEN API > REST 방식 선택
        - SOAP는 잘 사용 안 함
        - 기본 요청 URL의 주소가 .xml 또는 .json이라고 써있는데 .json이 경량의 데이터이므로 (i.e., 글자 수가 적어서 전송 속도가 더 빠름) 되도록이면 .json 추천 (.xml은 .html의 조상)
- `pandas`, `openpyxl`, `xlrd` library/package의 설치 필요
    - pandas: 통계 분석할 때 사용하는 대표적인 라이브러리 (open source library
    - openpyxl과 xlrd는 엑셀을 다루는 라이브러리

```shell
$ pip3 install --upgrade pandas
$ pip3 install --upgrade openpyxl
$ pip3 install --upgrade xlrd
```

## 1. 필요한 모듈 참조

In [1]:
# 필요한 모듈 참조
import requests
    # request로 갖고 온 데이터값이 JSON형태이면 dictionary로 변환하면 된다
    # request로 갖고 온 데이터값이 HTML + CSS 이면 BeautifulSoup라는 클래스를 통해 HTML + CSS 안에 있는 내용들을 추출해준다
import json
    # json 형태를 딕셔너리로 바꿔주는 라이브러리
import datetime as dt
from pandas import DataFrame

## 2. url 객체 생성 (`url 기본 형식 + 요청 인터페이스`)
- 요청 인터페이스 중 필수로 들어가야 하는 요청 변수들 (i.e., 파라미터들):
    - `key`: 발급받은 키 값 입력
    - `targetDt`: 조회하고자 하는 날짜를 yyyymmdd 형식으로 입력
### 1) `url 기본 형식` 객체 생성
- url 상 `get 파라미터`의 경우 `?` 다음에 넣고 여러개일 경우 중간에 `&`를 넣는다
- 문법 예시:
    - 'https://www.kobis ... .json`?`key={발급받은키}`&`targetDt={원하는날짜}'

In [2]:
# 온라인 상의 URL 기본 형식 (.json 확장자 사용)
urlFmt = "http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key={key}&targetDt={targetDt}"

### 2) `발급받은 API 연동 키 (key)` 객체 생성

In [3]:
# 발급받은 API 연동키
API_KEY = "6d2cf4aa96725383235c717f2e569f1e"

### 3) `날짜 (targetDt)` 객체 생성

In [4]:
# 하루 전 날짜 얻기
    # 집계 데이터 같은 경우 해당 날의 집계는 다음 날이 되어야 볼 수 있는 경우가 대부분이기에 하루 전 날짜 얻기
date = dt.datetime.now() - dt.timedelta(days = 1)
yesterday = date.strftime("%Y%m%d")
yesterday

'20231231'

### 4) 최종 url 객체 생성

In [5]:
# 최종 URL 얻기
url = urlFmt.format(key=API_KEY, targetDt=yesterday)
url

'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=6d2cf4aa96725383235c717f2e569f1e&targetDt=20231231'

## 3. 접속 객체 생성 및 접속 정보 설정

In [7]:
# 접속 객체 생성
session = requests.Session()

# 접속 정보 설정
session.headers.update({
    "Referer": "",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36"
})

## 4. 원하는 웹페이지에 접속

In [9]:
# 특정 웹 페이지에 접속
r = session.get(url)

# 접속에 실패한 경우
if r.status_code != 200:
    # 에러코드와 에러메시지 출력
    msg = "[%d Error] %s 에러가 발생함" % (r.status_code, r.reason)
    # 에러를 강제로 생성시킴
    raise Exception(msg)
        # 예외 처리 raise Exception()

## 5. json 형식을 딕셔너리로 변환

In [10]:
# 인코딩 형식 지정
r.encoding = "utf-8"

# 가져온 문자열을 딕셔너리로 변환
data = json.loads(r.text)
data

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스',
  'showRange': '20231231~20231231',
  'dailyBoxOfficeList': [{'rnum': '1',
    'rank': '1',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20203702',
    'movieNm': '노량: 죽음의 바다',
    'openDt': '2023-12-20',
    'salesAmt': '3579747977',
    'salesShare': '39.3',
    'salesInten': '830646347',
    'salesChange': '30.2',
    'salesAcc': '34030699992',
    'audiCnt': '349911',
    'audiInten': '80957',
    'audiChange': '30.1',
    'audiAcc': '3437846',
    'scrnCnt': '1767',
    'showCnt': '6296'},
   {'rnum': '2',
    'rank': '2',
    'rankInten': '0',
    'rankOldAndNew': 'OLD',
    'movieCd': '20212866',
    'movieNm': '서울의 봄',
    'openDt': '2023-11-22',
    'salesAmt': '3315941046',
    'salesShare': '36.4',
    'salesInten': '716908734',
    'salesChange': '27.6',
    'salesAcc': '115351048010',
    'audiCnt': '326481',
    'audiInten': '69887',
    'audiChange': '27.2',
    'audiAcc': '11854819',
    'scrnCnt': '

## 6. 원하는 부분만 추출
- 똑같은 딕셔너리를 원소로 갖는 리스트 추출

In [11]:
# 필요한 부분만 추출하기
dailyBoxOfficeList = data['boxOfficeResult']['dailyBoxOfficeList']
dailyBoxOfficeList

[{'rnum': '1',
  'rank': '1',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20203702',
  'movieNm': '노량: 죽음의 바다',
  'openDt': '2023-12-20',
  'salesAmt': '3579747977',
  'salesShare': '39.3',
  'salesInten': '830646347',
  'salesChange': '30.2',
  'salesAcc': '34030699992',
  'audiCnt': '349911',
  'audiInten': '80957',
  'audiChange': '30.1',
  'audiAcc': '3437846',
  'scrnCnt': '1767',
  'showCnt': '6296'},
 {'rnum': '2',
  'rank': '2',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20212866',
  'movieNm': '서울의 봄',
  'openDt': '2023-11-22',
  'salesAmt': '3315941046',
  'salesShare': '36.4',
  'salesInten': '716908734',
  'salesChange': '27.6',
  'salesAcc': '115351048010',
  'audiCnt': '326481',
  'audiInten': '69887',
  'audiChange': '27.2',
  'audiAcc': '11854819',
  'scrnCnt': '1654',
  'showCnt': '5748'},
 {'rnum': '3',
  'rank': '3',
  'rankInten': '0',
  'rankOldAndNew': 'OLD',
  'movieCd': '20236146',
  'movieNm': '신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김

## 7. DataFrame으로 결과표 생성

In [12]:
# 결과 표 생성하기
df = DataFrame(dailyBoxOfficeList)
df

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20203702,노량: 죽음의 바다,2023-12-20,3579747977,39.3,830646347,30.2,34030699992,349911,80957,30.1,3437846,1767,6296
1,2,2,0,OLD,20212866,서울의 봄,2023-11-22,3315941046,36.4,716908734,27.6,115351048010,326481,69887,27.2,11854819,1654,5748
2,3,3,0,OLD,20236146,신차원! 짱구는 못말려 더 무비 초능력 대결전 ~날아라 수제김밥~,2023-12-22,635306120,7.0,92280590,17,5506229689,64990,9052,16.2,568257,843,1725
3,4,4,0,OLD,20235735,아쿠아맨과 로스트 킹덤,2023-12-20,664014080,7.3,110828213,20,7459568772,64054,11058,20.9,722978,873,1738
4,5,5,0,OLD,20235596,트롤: 밴드 투게더,2023-12-20,290486901,3.2,48630655,20.1,3109594201,30940,5245,20.4,336931,674,1009
5,6,6,0,OLD,20235290,도티와 영원의 탑,2023-12-27,113512552,1.2,3649005,3.3,388161976,11972,311,2.7,41696,370,438
6,7,7,0,OLD,20234673,뽀로로 극장판 슈퍼스타 대모험,2023-12-13,111402091,1.2,10830582,10.8,2933356296,11396,930,8.9,321203,326,424
7,8,8,0,OLD,20234114,괴물,2023-11-29,96165674,1.1,14117753,17.2,3791967275,9235,1466,18.9,387428,187,266
8,9,9,0,OLD,20236045,"바다 탐험대 옥토넛 어보브 앤 비욘드: 버드, 옥토경보를 울려라!",2023-12-27,53802250,0.6,6104390,12.8,319879760,6404,642,11.1,38823,212,254
9,10,10,0,OLD,20236080,류이치 사카모토: 오퍼스,2023-12-27,46951068,0.5,4578765,10.8,286231031,4662,403,9.5,31197,221,303


## 8. 데이터 저장하기

In [13]:
# 엑셀로 저장
df.to_excel("박스오피스_순위_%s.xlsx" % yesterday)